In [1]:
import sys, os

# Tell Python to also look one folder up (where 'db' lives)
sys.path.append(os.path.abspath(".."))

# Optional: confirm it worked
print(" Project path added.")

 Project path added.


In [3]:
from db.connect_db import get_connection
import pandas as pd
from pathlib import Path

# Create connection
connection = get_connection()
cursor = connection.cursor()

print("Connected to StudyBuddy database.")

Connected to StudyBuddy database.


In [5]:
!pip install faker
from db.connect_db import get_connection
import pandas as pd
import random
from faker import Faker
from pathlib import Path
import os

# Initialize Faker
fake = Faker()

# Create DB connection
connection = get_connection()
cursor = connection.cursor()

print(" Connected to StudyBuddy database.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.3 MB/s eta 0:00:00a 0:00:01
 Connected to StudyBuddy database.


In [6]:
cursor.execute("SELECT college_id FROM Colleges;")
college_ids = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT major_id FROM Majors;")
major_ids = [row[0] for row in cursor.fetchall()]

print(f" Found {len(college_ids)} colleges and {len(major_ids)} majors.")


 Found 2076 colleges and 425 majors.


In [8]:
college_levels = ['Freshman', 'Sophomore', 'Junior', 'Senior', 'Graduate']

# University-like email domains
domains = ['@usm.edu', '@maine.edu', '@student.usm.edu', '@college.edu']

# Bio templates for more realism
bio_templates = [
    "Computer Science student passionate about app development and AI.",
    "Focused on machine learning, databases, and research innovation.",
    "Driven by solving real-world problems through code and design.",
    "Interested in data analysis, visualization, and systems security.",
    "Exploring software engineering and collaborative projects."
]

users_data = []

for _ in range(100):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = f"{first_name.lower()}.{last_name.lower()}{random.choice(domains)}"
    password_hash = fake.sha256()  # fake password hash
    college_level = random.choices(college_levels, weights=[25, 20, 20, 15, 20])[0]
    bio = random.choice(bio_templates)
    college_id = random.choice(college_ids)
    major_id = random.choice(major_ids)

    users_data.append((
        email,
        password_hash,
        first_name,
        last_name,
        college_level,
        bio,
        college_id,
        major_id
    ))

print(f" Generated {len(users_data)} realistic users.")


 Generated 100 realistic users.


In [9]:
insert_query = """
INSERT INTO Users (email, password_hash, first_name, last_name, college_level, bio, college_id, major_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

cursor.executemany(insert_query, users_data)
connection.commit()

print(" Successfully inserted 100 users into the Users table!")


 Successfully inserted 100 users into the Users table!


In [ ]:
# Create DataFrame
df_users = pd.DataFrame(users_data, columns=[
    "email", "password_hash", "first_name", "last_name",
    "college_level", "bio", "college_id", "major_id"
])

# Define the output path
current_dir = Path.cwd()
if current_dir.name == "inserts":
    output_dir = current_dir.parent / "data" / "Raw_data"
else:
    output_dir = current_dir / "data" / "Raw_data"

output_dir.mkdir(parents=True, exist_ok=True)

# File path
csv_path = output_dir / "users_raw.csv"

# Save the file
df_users.to_csv(csv_path, index=False, encoding="utf-8")
print(f" Exported users.")


📁 Exported users.


In [12]:
cursor.execute("SELECT COUNT(*) FROM Users;")
print(" Total users in table:", cursor.fetchone()[0])

cursor.execute("SELECT user_id, first_name, last_name, college_level FROM Users LIMIT 10;")
for row in cursor.fetchall():
    print(row)


 Total users in table: 100
(1, 'Angel', 'Clark', 'Freshman')
(2, 'Derrick', 'Wilson', 'Freshman')
(3, 'Jeffrey', 'Garrison', 'Sophomore')
(4, 'Richard', 'Rodriguez', 'Sophomore')
(5, 'Melissa', 'Jones', 'Freshman')
(6, 'Christopher', 'Lamb', 'Freshman')
(7, 'Jason', 'Lane', 'Freshman')
(8, 'Samantha', 'Berry', 'Graduate')
(9, 'Zachary', 'Burns', 'Sophomore')
(10, 'Alexander', 'Mosley', 'Sophomore')
